# Lecture1 Talk to Alpaca-LoRA

「データサイエンス特論2023」AIIT、岩政より。

- このノートブックは、Alplaca-LoRAという大規模言語モデル(LLM)を使った、チャットの例です。
- このノートブックは、Google Colabで、こんなことまでできるという最新の事例を紹介するために用意しました。
- Alplaca-LoRAの提供するCoalbファイルに、出力を日本語に翻訳する処理を追加しました(岩政)

【注意】Alpaca-LoRAは、メタ社のLLaMA-7Bに基づいています。商用にには利用できません。個人でのお試しとして使ってください。詳しくは、[ライセンス](https://huggingface.co/decapoda-research/llama-7b-hf/blob/main/LICENSE)を熟読のこと。



This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

最初に必要なモジュールをインポートします。

In [6]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git
!pip install transformers sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


モデル、transformerといいますを、準備します。高尾では、

In [2]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-1vx0k19x9kqgs --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [3]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [12]:
#FugMtによる英→日翻訳
from transformers import pipeline

# パイプラインの準備
ej_translator = pipeline("translation", model="staka/fugumt-en-ja")

# ここから、Alpaca-LoRa
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None, ejmode="en"):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        if ejmode=="ja":
          response= ej_translator(output.split("### Response:")[1].strip())
        else:
          response=output.split("### Response:")[1].strip()
        print("Response:", response)

/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


日本語でいきなり聞いてみましょう。

In [10]:
evaluate("江戸時代が２００年続いた理由を述べよ")

Response: The Edo Period lasted for 250 years, from 1603 to 1868. During this time, the Tokugawa shogunate was in power and the country experienced a period of relative peace and stability. This allowed for the development of a strong economy, the growth of cities, and the flourishing of arts and culture.


引数に、ejmode="ja"を入れると、出力を日本語に翻訳します。Thanks FuguMA

In [13]:
evaluate("江戸時代が２００年続いた理由を述べよ",ejmode="ja")

Response: [{'translation_text': '江戸時代は慶長8年(1603年)から慶応4年(1868年)まで200年間続き、この間、徳川幕府は比較的平和で安定的な時代を迎え、経済成長と新技術の発展、伝統工芸の保存を可能にした。'}]


In [ ]:
evaluate("Is the author of the ""Allice in wonderland"" a mathematician? tell me why")

Response: No, the author of Alice in Wonderland, Lewis Carroll, was not a mathematician. He was an English writer, logician, mathematician, and photographer.


会話はしてませんが、連続して入力するようにしてみました、会話ではないので、いったことはすぐに忘れます！

In [16]:
def main():
     while(True):
         print("You:", end="")
         message = input()
         print(evaluate(message,ejmode="ja"))

In [17]:
main()

You:最後の将軍はだれ？
Response: [{'translation_text': '最後の将軍は徳川慶喜。'}]
None
You:日本国憲法において、国民には何が保証される？
Response: [{'translation_text': '日本国憲法では、生活の権利、自由、幸福の追求など、特定の権利と自由が国民に保証されています。'}]
None
You:米国の大統領で、最も愚かだといわれているのは誰？
Response: [{'translation_text': '米国で最も悪名高い人物はドナルド・トランプです。'}]
None
You:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in main:4                                                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/ipykernel/kernelbase.py:851 in raw_input                  │
│                                                                                                  │
│   848 │   │   │   raise StdinNotImplementedError(                                                │
│   849 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   850 │   │   │   )                                                                              │
│ ❱ 851 │   │   return self._input_request(str(prompt),                                            │
│   852 │   │   │   self._parent_ident,                                                            │
│   853 │   │   │   self._parent_header,                                                           │
│   854 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/ipykernel/kernelbase.py:895 in _input_request             │
│                                                                                                  │
│   892 │   │   │   │   │   │   break                                                              │
│   893 │   │   │   except KeyboardInterrupt:                                                      │
│   894 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 895 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   896 │   │   │   except Exception as e:                                                         │
│   897 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   898                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user